### Linear Regression with ``BigDL-Keras``

In [1]:
#import packages
from __future__ import print_function
import numpy as np

In [2]:
#Define Parameters
num_inputs = 2
num_outputs = 1
num_examples = 10000
input_mean = [0, 0]
input_variance = [[1, 0],[0, 1]]
noise_mean = 0
noise_variance = 0.01
#Define function which return Y
def real_fn(X):
    return 2 * X[:, 0] - 3.4 * X[:, 1] + 4.2
    
X = np.random.multivariate_normal(input_mean, input_variance, num_examples)
X /= np.max(X)
noise = np.random.normal(noise_mean, noise_variance, num_examples)
Y = real_fn(X) + noise

In [3]:
#import bigdl package
from bigdl.nn.keras.layer import Dense
dense = Dense(1, input_dim = 2, name="Linear", bias=True)

/home/kunal/anaconda3/envs/deep_learning/lib/python3.6/site-packages/bigdl/util/engine.py:41: UserWarning: Find both SPARK_HOME and pyspark. You may need to check whether they match with each other. SPARK_HOME environment variable is set to: /home/kunal/Downloads/spark-2.4.7-bin-hadoop2.7/, and pyspark is found in: /home/kunal/anaconda3/envs/deep_learning/lib/python3.6/site-packages/pyspark/__init__.py. If they are unmatched, please use one source only to avoid conflict. For example, you can unset SPARK_HOME and use pyspark only.
  warnings.warn(warning_msg)


Prepending /home/kunal/anaconda3/envs/deep_learning/lib/python3.6/site-packages/bigdl/share/conf/spark-bigdl.conf to sys.path
creating: createKerasDense


In [4]:
#In BigDL Keras API, we use 'Sequential' as a container to store stacks of layers. Now that we created a dense layer,
#it's needed to be added into a Sequential constructor to finish defining a linear model.
from bigdl.nn.keras.topology import Sequential
linear_model = Sequential()
linear_model.add(dense)

creating: createKerasSequential


/home/kunal/anaconda3/envs/deep_learning/lib/python3.6/site-packages/bigdl/nn/keras/topology.py:213: UserWarning: bigdl.nn.keras is deprecated in 0.11. Recommend to use Analytics Zoo's Keras API.
  warnings.warn("bigdl.nn.keras is deprecated in 0.11. "


In [28]:
#Shape Inference
print(linear_model.get_input_shape())
print(linear_model.get_output_shape())
for layer in linear_model.flattened_layers(): print(layer)

(None, 2)
(None, 1)
Dense[Linear]


In [8]:
#Configure for training
linear_model.compile(loss='mse',
                  optimizer='sgd',
                  metrics=['accuracy'])

creating: createDefault
creating: createSGD
creating: createMSECriterion
creating: createTop1Accuracy


In [11]:
#Execute training
#distributed=True will indicate the training will be executed in a distributed mode
from multiprocessing import cpu_count
linear_model.fit(X, Y, batch_size = cpu_count()*2, nb_epoch=10, validation_data=None, distributed=True)